# awkward 配列の形状操作

本章では、Awkward Array における形状（階層構造）の操作方法を、`flatten` や `ravel`、規則軸（Regular）と非規則軸（Irregular）の変換、`unflatten` による層追加、配列結合、構造体配列の操作などを例示しながら解説します。

In [1]:
import numpy as np
import awkward as ak

## flatten と ravel

まず、多重にネストされた Awkward Array を用いて、`flatten()` がどの軸をどのように潰すかを確認します。

In [2]:
a = ak.Array([[[1, 2], None, [3]], [[None, 4], [5]], None, [[6, None]]])
a

<Array [[[1, 2], None, [3]], ..., [[6, ...]]] type='4 * option[var * option...'>

Awkward Array の `axis=0` での flatten は、最上位の None を取り除くだけです。

In [3]:
ak.flatten(a, axis=0)

<Array [[[1, 2], None, [3]], ..., [[6, ...]]] type='3 * var * option[var * ...'>

次に、`axis=1` や `axis=2` を指定すると、対応する層が削除され、要素が一段浅い層へ移動します。

In [4]:
ak.flatten(a, axis=1)

<Array [[1, 2], None, [3], ..., [5], [6, None]] type='6 * option[var * ?int64]'>

In [5]:
ak.flatten(a, axis=2)

<Array [[1, 2, 3], [None, ...], ..., [6, None]] type='4 * option[var * ?int64]'>

複数階層を削除したい場合は、`flatten` を複数回適用します。

In [6]:
ak.flatten(ak.flatten(a, axis=2), axis=1)

<Array [1, 2, 3, None, 4, 5, 6, None] type='8 * ?int64'>

`axis=None` を指定すると、最終軸のすべての値が一次元に取り出され、`None` は除去されます。

In [7]:
ak.flatten(a, axis=None)

<Array [1, 2, 3, 4, 5, 6] type='6 * int64'>

一方、`ravel()` は `flatten(None)` と似ていますが、`None` を保持します。

In [8]:
ak.ravel(a)

<Array [1, 2, 3, None, 4, 5, 6, None] type='8 * ?int64'>

## 規則軸と非規則軸

次に、Numpy 配列から生成した Awkward Array の規則軸（Regular）と、リストから生成した非規則軸（Irregular）の違いを確認します。

In [9]:
np.random.seed(42)
arr = np.random.randint(0, 10, (4, 6))
a = ak.Array(arr)
a

<Array [[6, 3, 7, 4, 6, 9], ..., [5, 1, 4, 0, 9, 5]] type='4 * 6 * int32'>

In [10]:
b = ak.Array(arr.tolist())
b

<Array [[6, 3, 7, 4, 6, 9], ..., [5, 1, 4, ..., 9, 5]] type='4 * var * int64'>

固定長の軸を可変長に変換します。

In [11]:
ak.from_regular(a, axis=1)

<Array [[6, 3, 7, 4, 6, 9], ..., [5, 1, 4, ..., 9, 5]] type='4 * var * int32'>

可変長軸を固定長軸へ変換します。

In [ ]:
ak.to_regular(b, axis=1)

規則軸では Numpy と同様のブールインデックスが可能です。

In [12]:
arr[arr >= 4]

array([6, 7, 4, 6, 9, 6, 7, 4, 7, 7, 5, 4, 7, 5, 4, 9, 5], dtype=int32)

In [13]:
a[a >= 4]

<Array [6, 7, 4, 6, 9, 6, 7, 4, ..., 7, 5, 4, 7, 5, 4, 9, 5] type='17 * int32'>

非規則軸では、ネスト構造を維持したままフィルタされます。

In [14]:
c = b[b >= 4]
c

<Array [[6, 7, 4, 6, 9], [6, ..., 7], ..., [5, 4, 9, 5]] type='4 * var * int64'>

非規則軸のため `to_regular()` するとエラーになります。

In [15]:
try:
    ak.to_regular(c)
except ValueError as ex:
    print(ex)

cannot convert to RegularArray because subarray lengths are not regular (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-50/awkward-cpp/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)


### 長さを揃えてから規則化

次のコードは、非規則な長さを持つ各部分配列の長さを揃えるための処理です。まず `ak.num(c)` で各部分配列の長さを取得し、その中の最大値を `ak.max(ak.num(c))` で求めます。その最大長を `target_length` として `ak.pad_none(c, target_length)` を適用することで、長さが足りない部分配列には末尾に `None` を追加してすべての部分配列の長さを統一し、後続で `ak.to_regular()` による規則軸への変換が可能な状態にします。

In [28]:
d = ak.pad_none(c, ak.max(ak.num(c)))
d

<Array [[6, 7, 4, 6, 9], ..., [5, 4, 9, 5, None]] type='4 * var * ?int64'>

In [18]:
ak.to_regular(d)

<Array [[6, 7, 4, 6, 9], [...], ..., [5, 4, 9, 5, None]] type='4 * 5 * ?int64'>

左側に None を入れたい場合：

In [19]:
ak.pad_none(c[:, ::-1], ak.max(ak.num(c)))[:, ::-1]

<Array [[6, 7, 4, 6, 9], ..., [None, 5, 4, 9, 5]] type='4 * var * ?int64'>

`clip`引数を`True`にすると、各部分配列の長さが指定した長さを超えている場合に、余分な要素を末尾から切り捨てて指定した長さに揃えます。次のコードでは、各部分配列の長さを `c` の最小長に合わせ、長すぎる部分配列は余分な要素を切り捨てることで、すべての部分配列の長さを同じにします。クリップされた軸は規則軸になります。

In [29]:
ak.pad_none(c, int(ak.min(ak.num(c))), clip=True)

<Array [[6, 7, 4, 6], [6, 7, ..., 7], ..., [5, 4, 9, 5]] type='4 * 4 * ?int64'>

### 新しい規則軸を追加

NumPyと同じように、`None` を使って新しい軸を追加できます。この場合、追加された軸の長さは 1 で、規則軸になります。

In [30]:
b[..., None]

<Array [[[6], [3], [7], [4], [6], [9]], ..., [...]] type='4 * var * 1 * int64'>

In [31]:
b[:, None, :]

<Array [[[6, 3, 7, 4, 6, 9]], ..., [[5, 1, ..., 5]]] type='4 * 1 * var * int64'>

`singletons()` を使うと、指定軸の後ろに長さ 1 の非規則軸が追加されます。

In [32]:
ak.singletons(a, axis=1)

<Array [[[6], [3], [7], [4], [6], [9]], ..., [...]] type='4 * 6 * var * int32'>

In [33]:
ak.singletons(a, axis=0)

<Array [[[6, 3, 7, 4, 6, 9]], ..., [[5, 1, ..., 5]]] type='4 * var * 6 * int32'>

## unflatten で層追加

`ak.unflatten()` は、一次元配列を指定した長さごとに分割して階層化されたリスト（部分配列）の形に変換する関数です。例えば次のコードは、最初の 3 要素 `[1, 2, 3]` が第一の部分配列、次の 2 要素 `[4, 5]` が第二の部分配列、長さ 0 の空配列が第三、最後の 3 要素 `[6, 7, 8]` が第四の部分配列として、それぞれネストされた配列として返されます。このように `unflatten` を使うことで、一次元配列を任意の長さごとに階層化して新しい階層構造を作ることができ、後続の処理で部分配列単位での操作が容易になります。


In [34]:
a = ak.Array([1, 2, 3, 4, 5, 6, 7, 8])
b = ak.unflatten(a, [3, 2, 0, 3])
b

<Array [[1, 2, 3], [4, 5], [], [6, 7, 8]] type='4 * var * int64'>

既存の階層に対しても、`axis` を指定することで新しい層を挿入できます。`ak.unflatten()` の第二引数は、分割したい各部分配列の長さを順番に指定する一次元配列です。この場合、`a` は既に二次元の配列（各行が部分配列）ですが、`axis=1` を指定することで、各行の中の要素をさらに指定した長さごとに分割して新しい階層を作ることができます。次のコードでは、第二引数 `[4, 2, 1, 2, 2, 1, 3]` が「各部分配列を分割する長さ」を一次元配列として順番に指定しており、最初の行 `[1,2,3,4,5,6]` は `[1,2,3,4]` と `[5,6]` に、次の行 `[1,2,3,4,5]` は `[1]` と `[2,3]`、最後の行 `[1,2,3,4]` は `[2,1,3]` のように、それぞれ指定された長さで部分配列に分割されます。こうすることで、既存の階層の内部に任意の長さの新しい階層（部分配列）を挿入することが可能になります。

In [37]:
a = ak.Array([[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5], [1, 2, 3, 4]])
ak.unflatten(a, [4, 2, 1, 2, 2, 1, 3], axis=1)

<Array [[[1, 2, 3, 4], [5, 6]], ..., [[1], ...]] type='3 * var * var * int64'>

0 軸に新しい層を追加する例：

In [38]:
ak.unflatten(a, [1, 2], axis=0)

<Array [[[1, 2, 3, 4, 5, 6]], [[...], ...]] type='2 * var * var * int64'>

## 配列の結合

`concatenate()` を使うと、軸方向に配列を結合できます。

デフォルト（axis=0）で結合：

In [39]:
a = ak.Array([1, 2, 3, 4])
b = ak.Array([2, 3, 4, 5])
ak.concatenate([a, b])

<Array [1, 2, 3, 4, 2, 3, 4, 5] type='8 * int64'>

1 軸方向に結合：

In [40]:
ak.concatenate([a[:, None], b[:, None]], axis=1)

<Array [[1, 2], [2, 3], [3, 4], [4, 5]] type='4 * 2 * int64'>

データ型が異なる場合は union 型として結合されます。

In [41]:
c = ak.Array([[1, 2, 3], [3, 4]])
ak.concatenate([a, c])

<Array [1, 2, 3, 4, [1, 2, 3], [3, 4]] type='6 * union[int64, var * int64]'>

規則軸と非規則軸をもつ配列を結合すると、非規則軸になります。

In [43]:
ak.concatenate([a[:, None], c])

<Array [[1], [2], [3], [4], [1, 2, 3], [3, 4]] type='6 * var * int64'>

In [44]:
ak.concatenate([c, a[:2, None]], axis=1)

<Array [[1, 2, 3, 1], [3, 4, 2]] type='2 * var * int64'>

## 構造体配列

辞書のリストも、リストの辞書も、どちらも同様に構造体配列として扱われます。

In [45]:
a = ak.Array({"x": [1, 2, 3, 4], "y": [2, 3, 4, 5]})
b = ak.Array(
    [
        {"vx": 10, "vy": 20},
        {"vx": 11, "vy": 21},
        {"vx": 12, "vy": 22},
        {"vx": 13, "vy": 23},
    ]
)
a

<Array [{x: 1, y: 2}, {...}, ..., {x: 4, y: 5}] type='4 * {x: int64, y: int64}'>

In [ ]:
b

### フィールドの追加・削除

辞書のように新しいフィールドを追加できます。

In [46]:
a['z'] = 0
a

<Array [{x: 1, y: 2, z: 0}, ..., {x: 4, ...}] type='4 * {x: int64, y: int64...'>

In [47]:
a['z'] = [3, 4, 5, 6]
a

<Array [{x: 1, y: 2, z: 3}, ..., {x: 4, ...}] type='4 * {x: int64, y: int64...'>

フィールド削除は `del a['z']` ですが、非破壊的に除外するには：

In [48]:
ak.without_field(a, 'z')

<Array [{x: 1, y: 2}, {...}, ..., {x: 4, y: 5}] type='4 * {x: int64, y: int64}'>

フィールド追加の非破壊版：

In [49]:
ak.with_field(a, np.sqrt(a.x**2 + a.y**2), 'r')

<Array [{x: 1, y: 2, z: 3, r: 2.24}, ...] type='4 * {x: int64, y: int64, z:...'>

### unzip でフィールドを展開

タプルとして出力：

In [50]:
ak.unzip(a)

(<Array [1, 2, 3, 4] type='4 * int64'>,
 <Array [2, 3, 4, 5] type='4 * int64'>,
 <Array [3, 4, 5, 6] type='4 * int64'>)

辞書として出力：

In [51]:
ak.unzip(a, how=dict)

{'x': <Array [1, 2, 3, 4] type='4 * int64'>,
 'y': <Array [2, 3, 4, 5] type='4 * int64'>,
 'z': <Array [3, 4, 5, 6] type='4 * int64'>}

### unzip + zip で構造体配列をマージ

In [52]:
ak.zip(ak.unzip(a, how=dict) | ak.unzip(b, how=dict))

<Array [{x: 1, y: 2, z: 3, vx: 10, ...}, ...] type='4 * {x: int64, y: int64...'>

### 構造体配列を2次元配列に変換

構造体配列を 2 次元配列へ展開する方法を示します。

In [53]:
array = ak.Array(
    [
        {"x": 11, "y": 12, "z": 13},
        {"x": 21, "y": 22, "z": 23},
        {"x": 31, "y": 32, "z": 33},
        {"x": 41, "y": 42, "z": 43},
        {"x": 51, "y": 52, "z": 53},
    ]
)
array

<Array [{x: 11, y: 12, z: 13}, ..., {...}] type='5 * {x: int64, y: int64, z...'>

一次元のレコードを二次元化：

In [54]:
array[:, None]

<Array [[{x: 11, y: 12, z: 13}], ..., [{...}]] type='5 * 1 * {x: int64, y: ...'>

各フィールドを取り出す：

In [55]:
ak.unzip(array[:, None])

(<Array [[11], [21], [31], [41], [51]] type='5 * 1 * int64'>,
 <Array [[12], [22], [32], [42], [52]] type='5 * 1 * int64'>,
 <Array [[13], [23], [33], [43], [53]] type='5 * 1 * int64'>)

取り出した値を結合して行ベクトルに：

In [56]:
ak.concatenate(ak.unzip(array[:, None]), axis=-1)

<Array [[11, 12, 13], [21, ..., 23], ..., [51, 52, 53]] type='5 * 3 * int64'>

`singletons()` でも同様の処理が可能：

In [57]:
ak.concatenate(ak.unzip(ak.singletons(array)), axis=-1)

<Array [[11, 12, 13], [21, ..., 23], ..., [51, 52, 53]] type='5 * var * int64'>